In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import  OrdinalEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve

import lightgbm as lgb
import xgboost as xgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import optuna

numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

train = pd.read_csv('./data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./data/test.csv').drop(columns=['ID'])

X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

# 수치형 컬럼들을 0으로 채움
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

# **PCA**로 차원 축소 후 군집화 수행 (군집 수는 3~6 사이로 실험)
pca = PCA(n_components=7, random_state=42)
X_pca = pca.fit_transform(X_train_encoded[numeric_columns])

kmeans = KMeans(n_clusters=5, random_state=42)
kmeans_clusters = kmeans.fit_predict(X_pca)

# 새로운 군집 피처 추가
X_train_encoded['cluster_group'] = kmeans_clusters
X_test_encoded['cluster_group'] = kmeans.predict(pca.transform(X_test_encoded[numeric_columns]))

# categorical_feature에 인덱스를 전달
categorical_feature_indices = [X_train_encoded.columns.get_loc(col) for col in categorical_columns]

# 스케일링 적용
scaler = StandardScaler()
X_train_encoded[numeric_columns] = scaler.fit_transform(X_train_encoded[numeric_columns])
X_test_encoded[numeric_columns] = scaler.transform(X_test_encoded[numeric_columns])

# # 데이터 불균형 확인 및 SMOTE + Tomek 적용
# smt = SMOTETomek(sampling_strategy=0.5, random_state=42)
# X_train_encoded, y_train = smt.fit_resample(X_train, y_train)

# # Feature Selection (SelectKBest)
# selector = SelectKBest(score_func=mutual_info_classif, k=40)  # 가장 중요한 40개의 변수를 선택
# X_train_encoded = selector.fit_transform(X_train_encoded, y)
# X_test_encoded = selector.transform(X_test_encoded)

print("-------------------------- 1. LightGBM 최적 파라미터로 모델 학습 --------------------------")
lgb_est = LGBMClassifier(
    class_weight='balanced',
    random_state=42,
    objective='binary',
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=70,
    max_depth=-1,
    device='gpu',  # GPU 사용
    gpu_platform_id=0,
    gpu_device_id=2
)

# LightGBM 모델 학습
lgb_est.fit(X_train_encoded, y)

# LightGBM으로 Feature Importance 확인
feature_importance = pd.DataFrame({
    'feature': X_train_encoded.columns,
    'importance': lgb_est.feature_importances_
}).sort_values(by='importance', ascending=False)

low_importance_features = feature_importance[feature_importance['importance'] <= 10]['feature'].tolist()
print("중요도가 낮은 피처:", low_importance_features)

# 중요도가 낮은 피처 제거
X_train_encoded = X_train_encoded.drop(columns=low_importance_features)
X_test_encoded = X_test_encoded.drop(columns=low_importance_features)

-------------------------- 1. LightGBM 최적 파라미터로 모델 학습 --------------------------
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 743
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 63
[LightGBM] [Info] Using requested OpenCL platform 0 device 2
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (4.89 MB) transferred to GPU in 0.004208 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
중요도가 낮은 피처: ['동결 배아 사용 여부', '불임 원인 - 정자 형태', '불임

In [ ]:


# def objective_lgb(trial):
#     params = {
#         'objective': 'binary',
#         'n_estimators': 1000,
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 31, 100),
#         'max_depth': trial.suggest_int('max_depth', -1, 20),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'device': 'gpu',
#         'gpu_platform_id': 0,
#         'gpu_device_id': 2
#     }
    
#     model = LGBMClassifier(**params)
#     model.fit(X_train_reduced, y_train)
#     y_pred = model.predict_proba(X_val_reduced)[:, 1]
#     return roc_auc_score(y_val, y_pred)

# study_lgb = optuna.create_study(direction='maximize')
# study_lgb.optimize(objective_lgb, n_trials=50)
# print("Best parameters for LightGBM:", study_lgb.best_params)


In [35]:
# def objective_xgb(trial):
#     params = {
#         'n_estimators': 300,
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
#         'max_depth': trial.suggest_int('max_depth', 3, 12),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'gamma': trial.suggest_float('gamma', 0, 5),
#         'tree_method': 'hist',  # 권장 방식
#         'device': 'cuda'
#     }
    
#     model = XGBClassifier(**params)
#     model.fit(X_train_reduced, y_train)
#     y_pred = model.predict_proba(X_val_reduced)[:, 1]
#     return roc_auc_score(y_val, y_pred)

# study_xgb = optuna.create_study(direction='maximize')
# study_xgb.optimize(objective_xgb, n_trials=50)
# print("Best parameters for XGBoost:", study_xgb.best_params)

In [36]:
# def objective_cat(trial):
#     params = {
#         'iterations': 1000,
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
#         'depth': trial.suggest_int('depth', 4, 12),
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
#         'random_seed': 42,
#         'task_type': 'GPU',
#         'devices': '0'
#     }
    
#     model = CatBoostClassifier(**params)
#     model.fit(X_train_reduced, y_train, verbose=0)
#     y_pred = model.predict_proba(X_val_reduced)[:, 1]
#     return roc_auc_score(y_val, y_pred)

# study_cat = optuna.create_study(direction='maximize')
# study_cat.optimize(objective_cat, n_trials=50)
# print("Best parameters for CatBoost:", study_cat.best_params)

In [ ]:
print("-------------------------- 1. LightGBM 최적 파라미터로 모델 학습 --------------------------")
# Best parameters for LightGBM: {'learning_rate': 0.010682507488382762, 'num_leaves': 49, 'max_depth': 17, 'subsample': 0.8215664737448534, 'colsample_bytree': 0.8497893445033722}
lgb_est = LGBMClassifier(
    class_weight='balanced',
    random_state=42,
    objective='binary',
    n_estimators=1000,
    subsample=0.82,
    colsample_bytree=0.85,
    learning_rate=0.01,
    num_leaves=49,
    max_depth=17,
    device='gpu',  # GPU 사용
    gpu_platform_id=0,
    gpu_device_id=2
)

# LightGBM 모델 학습
lgb_est.fit(X_train_reduced, y_train)
y_train_pred_lgb = lgb_est.predict_proba(X_val_reduced)[:, 1]
roc_auc_lgb = roc_auc_score(y_val, y_train_pred_lgb)

print("-------------------------- 2. XGBoost 최적 파라미터로 모델 학습 --------------------------")
# Best parameters for XGBoost: {'learning_rate': 0.027828379673739464, 'max_depth': 9, 'subsample': 0.8401352803766647, 'colsample_bytree': 0.817257623227272, 'gamma': 4.054300711621877}
xgb_est = XGBClassifier(
    eval_metric='auc',
    random_state=42,
    n_estimators=300,
    learning_rate=0.027,
    max_depth=9,
    subsample=0.84,
    colsample_bytree=0.82,
    gamma=4,
    tree_method='hist',  # 새 권장 방식
    device='cuda'        # GPU 사용
)

# XGBoost 모델 학습
xgb_est.fit(X_train_reduced, y_train)
y_train_pred_xgb = xgb_est.predict_proba(X_val_reduced)[:, 1]
roc_auc_xgb = roc_auc_score(y_val, y_train_pred_xgb)

print("-------------------------- 3. CatBoost 최적 파라미터로 모델 학습 --------------------------")
# Best parameters for CatBoost: {'learning_rate': 0.03222538120048102, 'depth': 6, 'l2_leaf_reg': 1.380048205342132}
catboost_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.032,
    depth=6,
    l2_leaf_reg=1.38,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    verbose=100,  # 경고 메시지 최소화
    task_type='GPU',
    devices='1'
)

# CatBoost 모델 학습
catboost_model.fit(X_train_reduced, y_train)
y_train_pred_cat = catboost_model.predict_proba(X_val_reduced)[:, 1]
roc_auc_cat = roc_auc_score(y_val, y_train_pred_cat)

def objective_weights(trial):
    w1 = trial.suggest_float('w1', 0, 1)
    w2 = trial.suggest_float('w2', 0, 1 - w1)
    w3 = 1 - w1 - w2
    
    final_pred_proba = w1 * y_train_pred_lgb + w2 * y_train_pred_xgb + w3 * y_train_pred_cat
    return roc_auc_score(y_val, final_pred_proba)

study_weights = optuna.create_study(direction='maximize')
study_weights.optimize(objective_weights, n_trials=50)
best_weights = study_weights.best_params
print(f"최적 가중치: LightGBM={best_weights['w1']:.2f}, XGBoost={best_weights['w2']:.2f}, CatBoost={1 - best_weights['w1'] - best_weights['w2']:.2f}")
# 최적 가중치: LightGBM=0.14, XGBoost=0.31, CatBoost=0.55
# 최적 가중치: LightGBM=0.18, XGBoost=0.34, CatBoost=0.48
# 최적 가중치: LightGBM=0.15, XGBoost=0.32, CatBoost=0.53
# 최적 가중치: LightGBM=0.14, XGBoost=0.34, CatBoost=0.52
final_pred_proba = 0.15 * y_train_pred_lgb + 0.33 * y_train_pred_xgb + 0.52 * y_train_pred_cat
roc_auc_final = roc_auc_score(y_val, final_pred_proba)

print(f"LightGBM 모델의 ROC-AUC 점수: {roc_auc_lgb:.4f}")
print(f"XGBoost 모델의 ROC-AUC 점수: {roc_auc_xgb:.4f}")
print(f"CatBoost 모델의 ROC-AUC 점수: {roc_auc_cat:.4f}")
print(f"Final 모델의 ROC-AUC 점수: {roc_auc_final:.4f}")
# print(f"StackingClassifier 모델의 ROC-AUC 점수: {ROC_AUC_stacking:.4f}")

In [ ]:
# LightGBM 모델의 ROC-AUC 점수: 0.7359
# XGBoost 모델의 ROC-AUC 점수: 0.7308
# CatBoost 모델의 ROC-AUC 점수: 0.7354
# StackingClassifier 모델의 ROC-AUC 점수: 0.7201

# SMOTETomek 적용 0.8
# LightGBM 모델의 ROC-AUC 점수: 0.7364 
# XGBoost 모델의 ROC-AUC 점수: 0.7320 ++++++++++++++++++++++++++++++++ 최고
# CatBoost 모델의 ROC-AUC 점수: 0.7361

# SMOTETomek 적용 0.5
# LightGBM 모델의 ROC-AUC 점수: 0.7366 ++++++++++++++++++++++++++++++++ 최고
# XGBoost 모델의 ROC-AUC 점수: 0.7317
# CatBoost 모델의 ROC-AUC 점수: 0.7363

# # 미적용
# LightGBM 모델의 ROC-AUC 점수: 0.7364
# XGBoost 모델의 ROC-AUC 점수: 0.7276
# CatBoost 모델의 ROC-AUC 점수: 0.7368
# Final 모델의 ROC-AUC 점수: 0.7359

# Q1-3 제거
# LightGBM 모델의 ROC-AUC 점수: 0.7364
# XGBoost 모델의 ROC-AUC 점수: 0.7292
# CatBoost 모델의 ROC-AUC 점수: 0.7369 ++++++++++++++++++++++++++++++++ 최고
# Final 모델의 ROC-AUC 점수: 0.7361

# # Feature engnieering 적용
# LightGBM 모델의 ROC-AUC 점수: 0.7362
# XGBoost 모델의 ROC-AUC 점수: 0.7299
# CatBoost 모델의 ROC-AUC 점수: 0.7368
# Final 모델의 ROC-AUC 점수: 0.7363

# KNMeans 적용 군집
# LightGBM 모델의 ROC-AUC 점수: 0.7363
# XGBoost 모델의 ROC-AUC 점수: 0.7301
# CatBoost 모델의 ROC-AUC 점수: 0.7368
# Final 모델의 ROC-AUC 점수: 0.7364 

# feature importance 상위 30개 변수만 사용
# LightGBM 모델의 ROC-AUC 점수: 0.7356
# XGBoost 모델의 ROC-AUC 점수: 0.7278
# CatBoost 모델의 ROC-AUC 점수: 0.7364
# Final 모델의 ROC-AUC 점수: 0.7355

# feature importance ['불임 원인 - 정자 형태', '불임 원인 - 정자 면역학적 요인', '난자 해동 경과일', '불임 원인 - 자궁경부 문제', '불임 원인 - 여성 요인', '난자 채취 경과일'] 제거
# LightGBM 모델의 ROC-AUC 점수: 0.7363
# XGBoost 모델의 ROC-AUC 점수: 0.7299
# CatBoost 모델의 ROC-AUC 점수: 0.7369
# Final 모델의 ROC-AUC 점수: 0.7364  

# kmeans 5개 군집 PCA
# LightGBM 모델의 ROC-AUC 점수: 0.7365
# XGBoost 모델의 ROC-AUC 점수: 0.7297
# CatBoost 모델의 ROC-AUC 점수: 0.7368
# Final 모델의 ROC-AUC 점수: 0.7369

# hyperparameter tuning
# LightGBM 모델의 ROC-AUC 점수: 0.7369
# XGBoost 모델의 ROC-AUC 점수: 0.7368
# CatBoost 모델의 ROC-AUC 점수: 0.7371
# Final 모델의 ROC-AUC 점수: 0.7375

In [42]:
print("-------------------------- 1. LightGBM 최적 파라미터로 모델 학습 --------------------------")
# Best parameters for LightGBM: {'learning_rate': 0.010682507488382762, 'num_leaves': 49, 'max_depth': 17, 'subsample': 0.8215664737448534, 'colsample_bytree': 0.8497893445033722}
lgb_est = LGBMClassifier(
    class_weight='balanced',
    random_state=42,
    objective='binary',
    n_estimators=1000,
    subsample=0.82,
    colsample_bytree=0.85,
    learning_rate=0.01,
    num_leaves=49,
    max_depth=17,
    device='gpu',  # GPU 사용
    gpu_platform_id=0,
    gpu_device_id=2
)

# LightGBM 모델 학습
lgb_est.fit(X_train_encoded, y)
y_train_pred_lgb = lgb_est.predict_proba(X_test_encoded)[:, 1]

print("-------------------------- 2. XGBoost 최적 파라미터로 모델 학습 --------------------------")
# Best parameters for XGBoost: {'learning_rate': 0.027828379673739464, 'max_depth': 9, 'subsample': 0.8401352803766647, 'colsample_bytree': 0.817257623227272, 'gamma': 4.054300711621877}
xgb_est = XGBClassifier(
    eval_metric='auc',
    random_state=42,
    n_estimators=300,
    learning_rate=0.027,
    max_depth=9,
    subsample=0.84,
    colsample_bytree=0.82,
    gamma=4,
    tree_method='hist',  # 새 권장 방식
    device='cuda'        # GPU 사용
)

# XGBoost 모델 학습
xgb_est.fit(X_train_encoded, y)
y_train_pred_xgb = xgb_est.predict_proba(X_test_encoded)[:, 1]

print("-------------------------- 3. CatBoost 최적 파라미터로 모델 학습 --------------------------")
# Best parameters for CatBoost: {'learning_rate': 0.03222538120048102, 'depth': 6, 'l2_leaf_reg': 1.380048205342132}
catboost_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.032,
    depth=6,
    l2_leaf_reg=1.38,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    verbose=100,  # 경고 메시지 최소화
    task_type='GPU',
    devices='1'
)

# CatBoost 모델 학습
catboost_model.fit(X_train_encoded, y)
y_train_pred_cat = catboost_model.predict_proba(X_test_encoded)[:, 1]

final_pred_proba = 0.1 * y_train_pred_lgb + 0.1 * y_train_pred_xgb + 0.8 * y_train_pred_cat

# 제출 파일 생성
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['probability'] = final_pred_proba
sample_submission.to_csv('./submit/final_ensemble_submit.csv', index=False)
print("최종 제출 파일 'final_ensemble_submit.csv' 생성 완료!")

-------------------------- 1. LightGBM 최적 파라미터로 모델 학습 --------------------------
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 738
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 61
[LightGBM] [Info] Using requested OpenCL platform 0 device 2
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (4.89 MB) transferred to GPU in 0.004743 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
-------------------------- 2. XGBoost 최적 파라미터로 모

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 4.71ms	remaining: 4.71s
100:	total: 374ms	remaining: 3.33s
200:	total: 726ms	remaining: 2.88s
300:	total: 1.08s	remaining: 2.5s
400:	total: 1.43s	remaining: 2.13s
500:	total: 1.78s	remaining: 1.77s
600:	total: 2.13s	remaining: 1.42s
700:	total: 2.48s	remaining: 1.06s
800:	total: 2.83s	remaining: 704ms
900:	total: 3.18s	remaining: 350ms
999:	total: 3.53s	remaining: 0us
최종 제출 파일 'final_ensemble_submit.csv' 생성 완료!


In [ ]:
# # 교차 검증 설정
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# cv_scores_lgb = []
# cv_scores_xgb = []
# cv_scores_stack = []
# cv_scores_catb = []

# # 교차 검증을 통한 ROC-AUC 비교
# for train_idx, val_idx in cv.split(X_train_encoded, y):
#     X_cv_train, X_cv_val = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[val_idx]
#     y_cv_train, y_cv_val = y.iloc[train_idx], y.iloc[val_idx]
    
#     # LightGBM 예측
#     lgb_est.fit(X_cv_train, y_cv_train)
#     lgb_proba = lgb_est.predict_proba(X_cv_val)[:, 1]
#     auc_lgb = roc_auc_score(y_cv_val, lgb_proba)
#     cv_scores_lgb.append(auc_lgb)
    
#     # XGBoost 예측
#     xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
#     xgb_model.fit(X_cv_train, y_cv_train)
#     xgb_proba = xgb_model.predict_proba(X_cv_val)[:, 1]
#     auc_xgb = roc_auc_score(y_cv_val, xgb_proba)
#     cv_scores_xgb.append(auc_xgb)

#     # catboost 예측
#     catboost_model.fit(X_cv_train, y_cv_train)
#     cat_proba = catboost_model.predict_proba(X_cv_val)[:, 1]
#     auc_catb = roc_auc_score(y_cv_val, cat_proba)
#     cv_scores_catb.append(auc_catb)
    
#     # Stacking 앙상블 예측
#     stacking_clf.fit(X_cv_train, y_cv_train)
#     stack_proba = stacking_clf.predict_proba(X_cv_val)[:, 1]
#     auc_stack = roc_auc_score(y_cv_val, stack_proba)
#     cv_scores_stack.append(auc_stack)

# # 평균 ROC-AUC 출력
# print("------------- 교차 검증 평균 ROC-AUC -------------")
# print(f"LightGBM: {np.mean(cv_scores_lgb):.4f}")
# print(f"XGBoost: {np.mean(cv_scores_xgb):.4f}")
# print(f"Stacking Ensemble: {np.mean(cv_scores_stack):.4f}")
# print("---------------------------------------------------")